In [30]:
import data
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
from torch.nn.init import normal_
data = pd.read_csv('data/movie.csv')

In [31]:
data

,user_id,movie_id,implicit_feedback
0,532,1044,0
1,667,812,0
2,230,229,0
3,899,829,0
4,91,412,0
...,...,...,...
199995,942,652,1
199996,942,672,1
199997,942,873,1
199998,942,935,1


In [34]:
data['user_id'].nunique()

943

In [35]:
data['movie_id'].nunique()

1682

In [17]:
dataset = TensorDataset(torch.LongTensor(np.array(data[['user_id', 'movie_id']])), torch.FloatTensor(np.array(data[['implicit_feedback']])))
dataset[0]

(tensor([ 532, 1044]), tensor([0.]))

In [18]:
dataLoader = DataLoader(dataset, batch_size=128, shuffle=True)
dataLoader

In [21]:
class MLPLayers(nn.Module):
    """
    여러 층의 MLP Layer Class
    
    Args:
        - layers: (List) input layer, hidden layer, output layer의 node 수를 저장한 List.
                ex) [5, 4, 3, 2] -> input layer: 5 nodes, output layer: 2 nodes, hidden layers: 4 nodes, 3 nodes
        - dropout: (float) dropout 확률
        - activation: (str) activation function의 함수. Default: 'relu'
    Shape:
        - Input: (torch.Tensor) input features. Shape: (batch size, # of input nodes)
        - Output: (torch.Tensor) output features. Shape: (batch size, # of output nodes)
    """
    def __init__(self, layers, dropout = 0):
        super(MLPLayers, self).__init__()
        
        # initialize Class attributes
        self.layers = layers
        self.n_layers = len(self.layers) - 1
        self.dropout = dropout
        
        # define layers
        mlp_modules = list()
        for i in range(self.n_layers):
            mlp_modules.append(nn.Dropout(p=self.dropout))
            input_size = self.layers[i]
            output_size = self.layers[i+1]
            mlp_modules.append(nn.Linear(input_size, output_size))
            mlp_modules.append(nn.ReLU())

        mlp_modules.append(nn.Linear(output_size, 1))
        self.mlp_layers = nn.Sequential(*mlp_modules)
        
        self.apply(self._init_weights)
        
    # initialize weights
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            normal_(module.weight.data, 0, 0.01)
            if module.bias is not None:
                module.bias.data.fill_(0.0)
    
    def forward(self, input_feature):
        return self.mlp_layers(input_feature)

In [ ]:
class NCF(nn.Module):
    """
    Neural Collaborative Filtering
    
    Args:
        - n_users: (int) 전체 유저의 수
        - n_items: (int) 전체 아이템의 수
        - emb_dim: (int) Embedding의 Dimension
        - layers: (List) Neural CF Layers의 각 node 수를 저장한 List.
                ex) [5, 4, 3, 2] -> hidden layers: 5 nodes, 4 nodes, 3 nodes, 2 nodes
        - dropout: (float) dropout 확률
    Shape:
        - Input: (torch.Tensor) input features, (user_id, item_id). Shape: (batch size, 2)
        - Output: (torch.Tensor) expected implicit feedback. Shape: (batch size,)
    """
    def __init__(self, n_users, n_items, emb_dim, layers, dropout):
        super(NCF, self).__init__()
        
        # initialize Class attributes
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.layers = layers
        self.n_layers = len(self.layers) + 1
        self.dropout = dropout
        
        # define layers
        self.user_embedding = nn.Embedding(self.n_users, self.emb_dim)# FILL HERE : USE nn.Embedding(단어의 개수, 임베딩 원하는 차원) # 
        self.item_embedding = nn.Embedding(self.n_items, self.emb_dim)# FILL HERE : USE nn.Embedding() #
        self.mlp_layers = MLPLayers([self.emb_dim*2] + self.layers, self.dropout) # FILL HERE : MLPLayers() #
        self.predict_layer = nn.Linear(self.layers[-1], 1) # FILL HERE : USE nn.Linear() #
        self.sigmoid = nn.Sigmoid()
        
        self.apply(self._init_weights)
        
    # initialize weights
    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            normal_(module.weight.data, mean=0.0, std=0.01)
        elif isinstance(module, nn.Linear):
            normal_(module.weight.data, 0, 0.01)
            if module.bias is not None:
                module.bias.data.fill_(0.0)
    
    def forward(self, input_feature):
        user, item = torch.split(input_feature, [1, 1], -1)
        user = user.squeeze(-1)
        item = item.squeeze(-1)
        
        user_e = self.user_embedding(user) # FILL HERE : USE self.user_embedding() #
        item_e = self.item_embedding(item) # FILL HERE : USE self.item_embedding() #
        input_feature = torch.cat((user_e, item_e), 1) # FILL HERE : USE torch.cat() #
        mlp_output = self.mlp_layers(input_feature) # FILL HERE : USE self.mlp_layers() #
        output = self.predict_layer(mlp_output) # FILL HERE : USE self.predict_layer() #
        output = self.sigmoid(output)
        return output.squeeze(-1)
    
    

In [25]:
model = MLPLayers([256, 64])
device = torch.device("cuda:{}".format(0) if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, amsgrad=True)
criterion = nn.BCELoss().to(device)

In [29]:
data.size()

torch.Size([128, 2])

In [26]:
model.train()

for batch_idx, (data, target) in enumerate(dataLoader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x2 and 256x64)